## Moat Brand Reporting Refresh

In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

In [14]:
from rtf_utils.moat_utils import MoatTile,moat_schemas
from essence.analytics.platform import securedcredentials as secure_creds
from rtf_utils.gcp_utils import BigQuery,CloudStorage
import warnings
from rtf_utils.bq_schema import moat_schema_dict
import json
import logging

In [5]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
warnings.filterwarnings("ignore", "Your application has authenticated using end user credentials")

In [6]:
service_account_email='131786951123-compute@developer.gserviceaccount.com' ## gcp analytics service account
PROJECT_ID='essence-analytics-dwh'

credentialsFromVault=secure_creds.getCredentialsFromEssenceVault(service_account_email)
print("Creds Loaded")
moat_token = secure_creds.getDataFromEssenceVault('Moat_Token_Google')
print("Token Loaded")

Creds Loaded
Token Loaded


In [8]:
## Define Moat Tasks for Stadia/Nest/Assistant
Q4 = [
    ## Google Disp
    (2506,{'level1':22783112},['date','level1','level3']), ##  stadia,
    (2506,{'level1':23221764},['date','level1','level3']), ##  nest 1,
    (2506,{'level1':23197607},['date','level1','level3']), ##  nest 2
    (2506,{'level1':23278016},['date','level1','level3']), ##  assistant
    (2506,{'level1':23219466},['date','level1','level3']), # Pixel
    
    ## Google Vid
   
    #(2698,{'level1':22783112},['date','level1','level3']), ## vid stadia,
    #(2698,{'level1':23221764},['date','level1','level3']), ## vid nest 1,
    #(2698,{'level1':23197607},['date','level1','level3']), ## vid nest 2,
    #(2698,{'level1':23278016},['date','level1','level3']), ## vid assistant
    
    ## YT TrueView ##
    (13386,{'level1':7492919005},['date','level1','level4']), # Stadia
    (13386,{'level1':7020493427},['date','level1','level4']),# Pixel
    ## YT Reserve 
    (6179366,{'level2':2604566125},['date','level2','level4']),## yt reserve - stadia
    
    ## YT Reserve DBM
    (6178833,{'level1':1533586},['date','level1','level4']),## yt res dbm  - assistant

    ## FB Display
    (6195503,{'level1':10154328017481183},['date','level1','level4']),#pixel
    ## FB Video NA
    #(6195505,{'level1':10154328017481183},['date','level1','level4']),#pixel need acccess
    ## FB Video Master
     (8268,{'level1':10154328017481183},,'date','level1','level4']),
    
    ## Twitter Display
    (6195541,{'level1':4503599660729481},['date','level1','level2']),#pixel
 ]

# need access to 6196284

In [8]:
def get_moat_data(tile_id,start_date, end_date,level_filters=None,dimensions=None,**kwargs):
    tile = MoatTile(tile_id,level_filters,dimensions)
    local_filename = tile.get_data(start_date,end_date,moat_token)
    return local_filename

def upload_local_file(local_filename,dest_bucket,folder=None):
    gcs = CloudStorage(credentialsFromVault)
    if folder:
        blob_name = folder + "/" + local_filename
    else:
        blob_name = local_filename
    blob = gcs.upload_blob(dest_bucket, blob_name,local_filename,mode='filename')
    return blob

# Prototype WorkFlow

In [9]:
import os
import time
os.environ['GOOGLE_CLOUD_PROJECT'] = 'essence-analytics-dwh'
#for x in bonito:
uploaded_blobs = []
for i,x in enumerate([(6195541,{'level1':4503599660729481},['date','level1','level2'])]):
    time.sleep(10)
    tile_id, level_filter, dimensions = x
    print("Get data for {]}".format(tile_id))
    filename = get_moat_data(tile_id,start_date,end_date,level_filter,dimensions)
    
    if filename:
        print("Stored at {}".format(filename))
    else:
        print("No Data")
        continue    
    
    try:
        print("Try GCS Upload")
        blob = upload_local_file(filename, bucket = "rtf_staging", folder = "brand_reporting")
        print("Clean Up File")
        uploaded_blobs.append(blob)
        os.remove(filename)
    except Exception as e:
        print(e)
        continue

INFO:root:API Request Time


6195541 {'level1': 4503599660729481} ['date', 'level1', 'level2']


INFO:root:6195541_4503599660729481.json Saved


Stored at 6195541_4503599660729481.json


INFO:root:File uploaded as brand_reporting/6195541_4503599660729481.json.


Clean Up File


In [50]:
file_uri = "gs://rtf_staging/brand_reporting/Brand Report - Upload File 2019-10-07T2046.csv"

dest_table = "003_DataLab_DailyPerformance"

bq.load_from_gcs("rtf_br_reporting",
                         file_uri,
                        dest_table)
                        #schema=schema_dict['schema']

In [12]:
bq = BigQuery(credentialsFromVault)
from google.cloud import bigquery

In [15]:
dataset = 'RTF_DWH_Moat'
for tile_id in [6179366,6195541,6195503]:
    print(tile_id)
    bq_schema = [bigquery.SchemaField.from_api_repr(x) for x in moat_schemas.get(tile_id)]

    load = bq.load_from_gcs(dataset, "gs://rtf_staging/brand_reporting/{}*".format(tile_id), "{}_20191001_20191020".format(tile_id),bq_schema,extension='json')
    print(load.job_id)

6179366
8448583f-c84a-4b60-a5f9-9d8c52554b9d
6195541
1e5ce770-5d85-427b-b96a-1f41a04ccf6a
6195503
529aa448-8a38-46e3-a19e-1fbeea77ca1c


In [70]:
from google.cloud import bigquery
bq_schema = []
for x in schema:
    bq_schema.append(bigquery.SchemaField(name=x.get('name'),field_type=x.get('type'), mode=x.get('mode')))

In [89]:
moat_schemas.get(2506)

[{'mode': 'REQUIRED', 'name': 'date', 'type': 'DATE'},
 {'mode': 'REQUIRED', 'name': 'level1_id', 'type': 'INTEGER'},
 {'mode': 'REQUIRED', 'name': 'level1_label', 'type': 'STRING'},
 {'mode': 'REQUIRED', 'name': 'level3_id', 'type': 'INTEGER'},
 {'mode': 'REQUIRED', 'name': 'level3_label', 'type': 'STRING'},
 {'mode': 'NULLABLE', 'name': 'loads_unfiltered', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'impressions_analyzed', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'human_and_viewable', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'valid_and_viewable', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'susp_human', 'type': 'FLOAT'},
 {'mode': 'NULLABLE', 'name': 'susp_valid', 'type': 'FLOAT'},
 {'mode': 'NULLABLE', 'name': 'moat_score', 'type': 'INTEGER'},
 {'mode': 'NULLABLE', 'name': 'iva', 'type': 'INTEGER'}]

In [122]:
bq_schema = [bigquery.SchemaField.from_api_repr(x) for x in moat_schemas.get(2506)]
load = bq.load_from_gcs(dataset, "gs://rtf_staging/brand_reporting/2506*", "2506_20191001_20191020",bq_schema,extension='json')